## מבוא

שיעור זה יכסה:
- מהי קריאת פונקציה ומקרי השימוש שלה
- כיצד ליצור קריאת פונקציה באמצעות OpenAI
- כיצד לשלב קריאת פונקציה באפליקציה

## מטרות הלמידה

לאחר סיום שיעור זה תדע כיצד ותבין:

- המטרה בשימוש בקריאת פונקציה
- הגדרת קריאת פונקציה באמצעות שירות OpenAI
- עיצוב קריאות פונקציה יעילות למקרי השימוש של האפליקציות שלך


## הבנת קריאות לפונקציות

בשיעור זה, אנו רוצים לבנות תכונה לסטארטאפ החינוכי שלנו שמאפשרת למשתמשים להשתמש בצ'אטבוט כדי למצוא קורסים טכניים. נמליץ על קורסים שמתאימים לרמת המיומנות שלהם, לתפקיד הנוכחי ולתחום הטכנולוגיה שמעניין אותם.

כדי להשלים זאת נשתמש בשילוב של:
 - `OpenAI` ליצירת חווית צ'אט למשתמש
 - `Microsoft Learn Catalog API` כדי לעזור למשתמשים למצוא קורסים בהתבסס על בקשת המשתמש
 - `Function Calling` כדי לקחת את השאילתה של המשתמש ולשלוח אותה לפונקציה שתבצע את בקשת ה-API.

כדי להתחיל, בואו נסתכל למה נרצה להשתמש בקריאת פונקציה מלכתחילה:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # לקבל תגובה חדשה מ-GPT שבה הוא יכול לראות את תגובת הפונקציה


print(second_response.choices[0].message)


### למה קריאת פונקציה

אם השלמתם כל שיעור אחר בקורס הזה, סביר להניח שאתם מבינים את הכוח שבשימוש במודלים לשוניים גדולים (LLMs). מקווים שגם תוכלו לראות כמה מהמגבלות שלהם.

קריאת פונקציה היא תכונה של שירות OpenAI שנועדה להתמודד עם האתגרים הבאים:

עיצוב תגובות לא עקבי:
- לפני קריאת פונקציה, התגובות ממודל שפה גדול היו לא מובנות ולא עקביות. המפתחים נאלצו לכתוב קוד אימות מורכב כדי להתמודד עם כל וריאציה בתוצאה.

אינטגרציה מוגבלת עם נתונים חיצוניים:
- לפני תכונה זו, היה קשה לשלב נתונים מחלקים אחרים של אפליקציה בהקשר של שיחה.

על ידי סטנדרטיזציה של פורמטי תגובה ואפשרות אינטגרציה חלקה עם נתונים חיצוניים, קריאת פונקציה מפשטת את הפיתוח ומפחיתה את הצורך בלוגיקת אימות נוספת.

משתמשים לא יכלו לקבל תשובות כמו "מה מזג האוויר הנוכחי בסטוקהולם?". זאת מכיוון שהמודלים היו מוגבלים לזמן שבו הנתונים אומנו.

בואו נסתכל על הדוגמה למטה שממחישה את הבעיה הזו:

נניח שאנחנו רוצים ליצור מסד נתונים של נתוני סטודנטים כדי שנוכל להציע להם את הקורס המתאים. למטה יש לנו שתי תיאורים של סטודנטים שהם מאוד דומים בנתונים שהם מכילים.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

אנחנו רוצים לשלוח את זה ל-LLM כדי לנתח את הנתונים. זה יכול לשמש מאוחר יותר באפליקציה שלנו כדי לשלוח את זה ל-API או לאחסן את זה בבסיס נתונים.

בואו ניצור שני פרומפטים זהים שבהם ננחה את ה-LLM לגבי המידע שמעניין אותנו:


אנחנו רוצים לשלוח את זה ל-LLM כדי לנתח את החלקים החשובים למוצר שלנו. כך נוכל ליצור שני פרומפטים זהים להנחות את ה-LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


לאחר יצירת שני ההנחיות הללו, נשלח אותן ל-LLM באמצעות `openai.ChatCompletion`. אנו מאחסנים את ההנחיה במשתנה `messages` ומקצים את התפקיד ל-`user`. זאת כדי לחקות הודעה ממשתמש שנכתבת לצ'אטבוט.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

עכשיו אנחנו יכולים לשלוח את שתי הבקשות ל-LLM ולבחון את התגובה שנקבל.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

למרות שההנחיות זהות והתיאורים דומים, אנו יכולים לקבל פורמטים שונים של המאפיין `Grades`.

אם תריץ את התא שלמעלה מספר פעמים, הפורמט יכול להיות `3.7` או `3.7 GPA`.

זה נובע מכך ש-LLM מקבל נתונים לא מובנים בצורת ההנחיה הכתובה ומחזיר גם נתונים לא מובנים. אנו צריכים פורמט מובנה כדי לדעת למה לצפות בעת אחסון או שימוש בנתונים אלו.

על ידי שימוש בקריאה פונקציונלית, אנו יכולים לוודא שאנו מקבלים חזרה נתונים מובנים. כאשר משתמשים בקריאה פונקציונלית, ה-LLM בפועל לא קורא או מריץ פונקציות כלשהן. במקום זאת, אנו יוצרים מבנה שה-LLM צריך לעקוב אחריו בתגובותיו. לאחר מכן אנו משתמשים בתגובות המובנות הללו כדי לדעת איזו פונקציה להריץ באפליקציות שלנו.


![דיאגרמת זרימת קריאת פונקציה](../../../../translated_images/Function-Flow.083875364af4f4bb.he.png)


אנו יכולים אז לקחת את מה שמוחזר מהפונקציה ולשלוח זאת חזרה ל-LLM. ה-LLM יגיב אז בשפה טבעית כדי לענות על שאלת המשתמש.


### מקרים לשימוש בקריאות לפונקציות

**קריאה לכלים חיצוניים**  
צ'אטבוטים מצוינים במתן תשובות לשאלות משתמשים. באמצעות קריאת פונקציות, הצ'אטבוטים יכולים להשתמש בהודעות מהמשתמשים כדי להשלים משימות מסוימות. לדוגמה, סטודנט יכול לבקש מהצ'אטבוט "שלח מייל למרצה שלי ואמר שאני צריך עזרה נוספת בנושא הזה". זה יכול לבצע קריאת פונקציה ל-`send_email(to: string, body: string)`

**יצירת שאילתות API או מסד נתונים**  
משתמשים יכולים למצוא מידע באמצעות שפה טבעית שהופכת לשאילתה מעוצבת או בקשת API. דוגמה לכך יכולה להיות מורה שמבקש "מי התלמידים שסיימו את המשימה האחרונה" שיכולה לקרוא לפונקציה בשם `get_completed(student_name: string, assignment: int, current_status: string)`

**יצירת נתונים מובנים**  
משתמשים יכולים לקחת בלוק טקסט או CSV ולהשתמש ב-LLM כדי לחלץ מידע חשוב ממנו. לדוגמה, סטודנט יכול להמיר מאמר מוויקיפדיה על הסכמי שלום ליצירת כרטיסיות AI ללמידה. זה יכול להיעשות באמצעות פונקציה בשם `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. יצירת קריאת הפונקציה הראשונה שלך

תהליך יצירת קריאת פונקציה כולל 3 שלבים עיקריים:  
1. קריאה ל-Chat Completions API עם רשימת הפונקציות שלך והודעת משתמש  
2. קריאת תגובת המודל לביצוע פעולה כלומר ביצוע פונקציה או קריאת API  
3. ביצוע קריאה נוספת ל-Chat Completions API עם התגובה מהפונקציה שלך כדי להשתמש במידע זה ליצירת תגובה למשתמש.


![זרימת קריאת פונקציה](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.he.png)


### אלמנטים של קריאת פונקציה

#### קלט משתמשים

השלב הראשון הוא ליצור הודעת משתמש. ניתן להקצות זאת באופן דינמי על ידי לקיחת הערך של שדה טקסט או ניתן להקצות ערך כאן. אם זו הפעם הראשונה שלך לעבוד עם ה-Chat Completions API, עלינו להגדיר את ה-`role` ואת ה-`content` של ההודעה.

ה-`role` יכול להיות או `system` (יצירת כללים), `assistant` (המודל) או `user` (המשתמש הקצה). עבור קריאת פונקציה, נקצה זאת כ-`user` ודוגמה לשאלה.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### יצירת פונקציות.

בהמשך נגדיר פונקציה ואת הפרמטרים של אותה פונקציה. נשתמש כאן בפונקציה אחת בלבד שנקראת `search_courses` אבל ניתן ליצור מספר פונקציות.

**חשוב**: פונקציות נכללות בהודעת המערכת ל-LLM וייכללו בכמות הטוקנים הזמינה שיש לך.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**הגדרות**

מבנה הגדרת הפונקציה כולל רמות מרובות, כל אחת עם התכונות שלה. הנה פירוט של המבנה המקונן:

**תכונות פונקציה ברמה העליונה:**

`name` - שם הפונקציה שאנו רוצים שתקרא.

`description` - זוהי התיאור של אופן פעולת הפונקציה. כאן חשוב להיות מדויק וברור.

`parameters` - רשימת ערכים ופורמט שברצונך שהמודל יפיק בתגובתו.

**תכונות אובייקט הפרמטרים:**

`type` - סוג הנתונים של אובייקט הפרמטרים (בדרך כלל "object").

`properties` - רשימת הערכים הספציפיים שהמודל ישתמש בהם בתגובתו.

**תכונות פרמטר יחיד:**

`name` - מוגדר במפורש על ידי מפתח התכונה (למשל, "role", "product", "level").

`type` - סוג הנתונים של פרמטר זה (למשל, "string", "number", "boolean").

`description` - תיאור הפרמטר הספציפי.

**תכונות אופציונליות:**

`required` - מערך המפרט אילו פרמטרים נדרשים כדי שהקריאה לפונקציה תושלם.


### ביצוע קריאת הפונקציה  
לאחר שהגדרנו פונקציה, כעת עלינו לכלול אותה בקריאה ל-Chat Completion API. אנו עושים זאת על ידי הוספת `functions` לבקשה. במקרה זה `functions=functions`.  

קיימת גם אפשרות להגדיר את `function_call` ל-`auto`. משמעות הדבר היא שנאפשר ל-LLM להחליט איזו פונקציה יש לקרוא בהתבסס על הודעת המשתמש במקום להקצות זאת בעצמנו.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

עכשיו בואו נסתכל על התגובה ונראה כיצד היא מעוצבת:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

ניתן לראות ששמו של הפונקציה נקרא וממסר המשתמש, ה-LLM הצליח למצוא את הנתונים המתאימים לארגומנטים של הפונקציה.


## 3.שילוב קריאות פונקציה באפליקציה. 


לאחר שבדקנו את התגובה המעוצבת מה-LLM, כעת נוכל לשלב זאת באפליקציה. 

### ניהול הזרימה 

כדי לשלב זאת באפליקציה שלנו, ננקוט בצעדים הבאים: 

ראשית, נבצע את הקריאה לשירותי OpenAI ונאחסן את ההודעה במשתנה בשם `response_message`. 


In [ ]:
response_message = response.choices[0].message

כעת נגדיר את הפונקציה שתתקשר ל-API של Microsoft Learn כדי לקבל רשימת קורסים:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



כפרקטיקה מומלצת, נבדוק לאחר מכן אם המודל רוצה לקרוא לפונקציה. לאחר מכן, ניצור אחת מהפונקציות הזמינות ונתאים אותה לפונקציה שנקראת.  
לאחר מכן, ניקח את הפרמטרים של הפונקציה ונמפה אותם לפרמטרים מה-LLM.

לבסוף, נוסיף את הודעת קריאת הפונקציה ואת הערכים שהוחזרו מהודעת `search_courses`. זה נותן ל-LLM את כל המידע שהוא צריך כדי  
להגיב למשתמש בשפה טבעית.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



כעת נשלח את ההודעה המעודכנת ל-LLM כדי שנוכל לקבל תגובה בשפה טבעית במקום תגובת JSON בפורמט API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## אתגר קוד

עבודה מצוינת! כדי להמשיך בלמידתך של קריאת פונקציות OpenAI תוכל לבנות: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - פרמטרים נוספים של הפונקציה שעשויים לעזור ללומדים למצוא קורסים נוספים. ניתן למצוא את פרמטרי ה-API הזמינים כאן:  
 - צור קריאת פונקציה נוספת שלוקחת מידע נוסף מהלומד כמו שפת האם שלו  
 - צור טיפול בשגיאות כאשר קריאת הפונקציה ו/או קריאת ה-API לא מחזירה קורסים מתאימים כלשהם  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון כי תרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור שלו נחשב למקור הסמכותי. למידע קריטי מומלץ להשתמש בתרגום מקצועי על ידי אדם. אנו לא נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
